In [ ]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from catboost import Pool, CatBoostClassifier
from lightgbm import LGBMClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [ ]:
text_features = ['premise','hypothesis']
# categorical_features = ['lang_abv']

y = train['label']
train.drop(['id','language','label','lang_abv'],axis=1,inplace=True)
# train['lang_abv'] = pd.factorize(train['lang_abv'])[0]

test.drop(['id','language','lang_abv'],axis=1,inplace=True)
# test['lang_abv'] = pd.factorize(test['lang_abv'])[0]

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
# No folds
# X_train, X_test, y_train, y_test = train_test_split(train,y,test_size=0.2)
# train_pool = Pool(
#     X_train, 
#     y_train, 
# #     cat_features=categorical_features, 
#     text_features=text_features, 
#     feature_names=list(X_train.columns)
# )
# valid_pool = Pool(
#     X_test, 
#     y_test,
# #     cat_features=categorical_features, 
#     text_features=text_features, 
#     feature_names=list(X_train.columns)
# )

# catboost_params = {
#     'iterations': 1000,
#     'learning_rate': 0.1,
#     'eval_metric': 'Accuracy',
#     'task_type': 'GPU',
#     'early_stopping_rounds': 100,
#     'use_best_model': True,
#     'verbose': 5000
# }
# model = CatBoostClassifier(**catboost_params)
# model.fit(train_pool, eval_set=valid_pool)

In [ ]:
# with folds
y_preds = []
models = []
oof_train = np.zeros((len(train),3))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

catboost_params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'eval_metric': 'Accuracy',
    'task_type': 'GPU',
    'early_stopping_rounds': 100,
    'use_best_model': True,
    'verbose': 100
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(train, y)):
    X_tr = train.loc[train_index, :]
    X_val = train.loc[valid_index, :]
    y_tr = y[train_index]
    y_val = y[valid_index]

    train_pool = Pool(
        X_tr, 
        y_tr, 
#         cat_features=categorical_cols,
        text_features=text_features,
        feature_names=list(X_tr)
    )
    valid_pool = Pool(
        X_val, 
        y_val, 
#         cat_features=categorical_cols,
        text_features=text_features,
        feature_names=list(X_tr)
    )

    model = CatBoostClassifier(**catboost_params)
    model.fit(train_pool, eval_set=valid_pool)

    oof_train[valid_index] = model.predict_proba(X_val)

    y_pred = model.predict_proba(test)
    y_preds.append(y_pred)
    models.append(model)

In [ ]:
lgbm = LGBMClassifier()
lgbm.fit(oof_train,y)

In [ ]:
pd.Series(np.argmax(y_pred,axis=1)).value_counts()

In [ ]:
pd.Series(lgbm.predict(np.mean(y_preds,axis=0))).value_counts()

In [ ]:
(lgbm.predict(np.mean(y_preds,axis=0)) == np.argmax(np.mean(y_preds,axis=0),axis=1)).sum()/len(test)

In [ ]:
(np.argmax(oof_train,axis=1) == y).sum()/len(y)

In [ ]:
submission = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/sample_submission.csv')

In [ ]:
submission['prediction'] = lgbm.predict(np.mean(y_preds,axis=0))

In [ ]:
submission['prediction'].value_counts()

In [ ]:
submission.to_csv('submission.csv', index = False)